# Генерація генома

In [2]:
#Ген
import random
class Gene:
    def __init__(self):
        self.identifier = random.random()
        self.owner = "owner"
        self.order = 0
    def __str__(self):
        line = "%s[%d]=%f "%(self.owner, self.order, self.identifier)
        return line

def mkGene(owner: str, order: int):
    g = Gene()
    g.owner = owner
    g.order = order
    return g

In [3]:
#Хромосома
import copy
class Chromosome:
    def __init__(self):
        self.genes = []
        self.pair = -1
    def __str__(self):
        lines = "Хромосома з %d пари. На %d генів\n"%(self.pair, len(self.genes))
        num_to_print = min(5, len(self.genes))
        for i in range(0, num_to_print):
            lines += str(self.genes[i])
        lines += "..."
        lines += str(self.genes[-1])
        lines += "\n"
        return lines

def mkChromosome(pair_id: int, owner: str, num_genes: int):
    c = Chromosome()
    c.pair = pair_id
    for i in range(0, num_genes):
        c.genes.append(mkGene(owner, i))
    return c

In [22]:
#Геном
class Genome:
    def __init__(self):
        self.chromosomes = []
    def __str__(self):
        lines = "Геном на %d хромосоми\n"%(len(self.chromosomes))
        for chromosome in self.chromosomes:
            lines += str(chromosome)
        lines += "...\n"
        return lines
        
def mkGenome(owner: str, chromosome_lengths: int):
    g = Genome()
    for chromosome_id in range(0, len(chromosome_lengths)):
        g.chromosomes.append(mkChromosome(chromosome_id, owner, chromosome_lengths[chromosome_id]))
        g.chromosomes.append(mkChromosome(chromosome_id, owner, chromosome_lengths[chromosome_id]))
    return g

In [23]:
#Зробимо жабок
chromosome = [200, 300, 400] #Конфігурація пар хромосом жабки
frog1 = mkGenome("Frog1", chromosome)
frog2 = mkGenome("Frog2", chromosome)
print(frog1)
print(frog2)

Геном на 6 хромосоми
Хромосома з 0 пари. На 200 генів
Frog1[0]=0.675253 Frog1[1]=0.315860 Frog1[2]=0.136631 Frog1[3]=0.045811 Frog1[4]=0.400276 ...Frog1[199]=0.942228 
Хромосома з 0 пари. На 200 генів
Frog1[0]=0.422981 Frog1[1]=0.471501 Frog1[2]=0.323627 Frog1[3]=0.408860 Frog1[4]=0.482989 ...Frog1[199]=0.446217 
Хромосома з 1 пари. На 300 генів
Frog1[0]=0.677181 Frog1[1]=0.336475 Frog1[2]=0.372219 Frog1[3]=0.748341 Frog1[4]=0.986958 ...Frog1[299]=0.601624 
Хромосома з 1 пари. На 300 генів
Frog1[0]=0.827092 Frog1[1]=0.939233 Frog1[2]=0.377816 Frog1[3]=0.511896 Frog1[4]=0.692375 ...Frog1[299]=0.843201 
Хромосома з 2 пари. На 400 генів
Frog1[0]=0.344375 Frog1[1]=0.774539 Frog1[2]=0.303013 Frog1[3]=0.553613 Frog1[4]=0.652588 ...Frog1[399]=0.907282 
Хромосома з 2 пари. На 400 генів
Frog1[0]=0.742951 Frog1[1]=0.728846 Frog1[2]=0.551823 Frog1[3]=0.621288 Frog1[4]=0.585013 ...Frog1[399]=0.642459 
...

Геном на 6 хромосоми
Хромосома з 0 пари. На 200 генів
Frog2[0]=0.042340 Frog2[1]=0.515805 Fr

# Кросінг овер

In [24]:
import math

npoints = 2 #Скільки точок у кросінговеру

#Функція яка вибирає точки кросінговеру
def crossover_points(pts, length):
    rands = []
    for i in range(0, pts):
        while True:
            point = math.floor(random.random() * length)
            if point in rands:
                continue
            else:
                rands.append(point)
                break
    rands = sorted(rands)
    ranges = []
    r = {}
    r["min"] = 0
    for point in rands:
        r["max"] = point
        ranges.append(r)
        r = {}
        r["min"] = point
    r["max"] = length
    ranges.append(r)
    return ranges
        
#Функція що виконує кросінговер двох хромосом
def crossover_chromo(ch1: Chromosome, ch2: Chromosome):
    if ch1.pair != ch2.pair:
        raise Exception("Хромосоми не з одної пари")
    length = min(len(ch1.genes), len(ch2.genes))
    
    ranges = crossover_points(npoints, length)
    first = True
    res_genes1 = []
    res_genes2 = []
    for r in ranges:
        if first == True:
            res_genes1 += ch1.genes[r["min"]:r["max"]]
            res_genes2 += ch2.genes[r["min"]:r["max"]]
            first = False
        else:
            res_genes1 += ch2.genes[r["min"]:r["max"]]
            res_genes2 += ch1.genes[r["min"]:r["max"]]
            first = True

    #Якщо хромосоми нерівні
    if length < len(ch1.genes):
        res_genes1 += ch1.genes[ranges[-1]["max"]:-1]
        
    if length < len(ch2.genes):
        res_genes2 += ch2.genes[ranges[-1]["max"]:-1]
    
    #Створюєм дві нові хромосоми
    new_ch1 = Chromosome()
    new_ch2 = Chromosome()
    new_ch1.pair = ch1.pair
    new_ch2.pair = ch2.pair
    new_ch1.genes = copy.deepcopy(res_genes1)
    new_ch2.genes = copy.deepcopy(res_genes2)
    
    #Перевіримо чи все правильно зроблено
    if len(res_genes1) != len(ch1.genes):
        raise Exception("Не співпадають довжини генів %d vs %d"%(len(ch1.genes), len(res_genes1)))
    if len(res_genes2) != len(ch2.genes):
        raise Exception("Не співпадають довжини генів %d vs %d"%(len(ch2.genes), len(res_genes2)))
    return (new_ch1, new_ch2)
    

In [25]:
(new_ch1, new_ch2) = crossover_chromo(frog1.chromosomes[0], frog1.chromosomes[1])

print("Стара хромосома 1")
print(frog1.chromosomes[0])
print("Нова хромосома 1")
print(new_ch1)

print("Стара хромосома 2")
print(frog1.chromosomes[1])
print("Нова хромосома 2")
print(new_ch2)

Стара хромосома 1
Хромосома з 0 пари. На 200 генів
Frog1[0]=0.675253 Frog1[1]=0.315860 Frog1[2]=0.136631 Frog1[3]=0.045811 Frog1[4]=0.400276 ...Frog1[199]=0.942228 

Нова хромосома 1
Хромосома з 0 пари. На 200 генів
Frog1[0]=0.675253 Frog1[1]=0.315860 Frog1[2]=0.136631 Frog1[3]=0.045811 Frog1[4]=0.400276 ...Frog1[199]=0.942228 

Стара хромосома 2
Хромосома з 0 пари. На 200 генів
Frog1[0]=0.422981 Frog1[1]=0.471501 Frog1[2]=0.323627 Frog1[3]=0.408860 Frog1[4]=0.482989 ...Frog1[199]=0.446217 

Нова хромосома 2
Хромосома з 0 пари. На 200 генів
Frog1[0]=0.422981 Frog1[1]=0.471501 Frog1[2]=0.323627 Frog1[3]=0.408860 Frog1[4]=0.482989 ...Frog1[199]=0.446217 



# Приготування половинного набору хромосом

In [33]:
#Функція що виконує мейоз
def meiosis(person: Genome):
    chromosome_pairs = sorted(list(set(map(lambda x: x.pair, person.chromosomes))))
    haplo1 = Genome()
    haplo2 = Genome()
    for p in chromosome_pairs:
        pair = list(filter(lambda x: x.pair == p, person.chromosomes))
        if len(pair) != 2:
            raise Exception("Chromosome pair not exist")
        (new_ch1, new_ch2) = crossover_chromo(pair[0], pair[1])
        haplo1.chromosomes.append(new_ch1)
        haplo2.chromosomes.append(new_ch2)
    return (haplo1, haplo2)

In [34]:
(h1, h2) = meiosis(frog1)
print("Повний набір")
print(frog1)
print("Половинний набір 1")
print(h1)
print("Половинний набір 2")
print(h2)

Повний набір
Геном на 6 хромосоми
Хромосома з 0 пари. На 200 генів
Frog1[0]=0.675253 Frog1[1]=0.315860 Frog1[2]=0.136631 Frog1[3]=0.045811 Frog1[4]=0.400276 ...Frog1[199]=0.942228 
Хромосома з 0 пари. На 200 генів
Frog1[0]=0.422981 Frog1[1]=0.471501 Frog1[2]=0.323627 Frog1[3]=0.408860 Frog1[4]=0.482989 ...Frog1[199]=0.446217 
Хромосома з 1 пари. На 300 генів
Frog1[0]=0.677181 Frog1[1]=0.336475 Frog1[2]=0.372219 Frog1[3]=0.748341 Frog1[4]=0.986958 ...Frog1[299]=0.601624 
Хромосома з 1 пари. На 300 генів
Frog1[0]=0.827092 Frog1[1]=0.939233 Frog1[2]=0.377816 Frog1[3]=0.511896 Frog1[4]=0.692375 ...Frog1[299]=0.843201 
Хромосома з 2 пари. На 400 генів
Frog1[0]=0.344375 Frog1[1]=0.774539 Frog1[2]=0.303013 Frog1[3]=0.553613 Frog1[4]=0.652588 ...Frog1[399]=0.907282 
Хромосома з 2 пари. На 400 генів
Frog1[0]=0.742951 Frog1[1]=0.728846 Frog1[2]=0.551823 Frog1[3]=0.621288 Frog1[4]=0.585013 ...Frog1[399]=0.642459 
...

Половинний набір 1
Геном на 3 хромосоми
Хромосома з 0 пари. На 200 генів
Frog1[

# Створення нової особи

In [35]:
#Функція відтворення
def reproduce(p1: Genome, p2: Genome):
    (p1h1, p1h2) = meiosis(p1)
    (p2h1, p2h2) = meiosis(p2)

    #Випадково вибрати гамету
    gameta1 = p1h1 if 0.5 > random.random() else p1h2
    gameta2 = p2h1 if 0.5 > random.random() else p2h2
    
    res = Genome()
    if len(gameta1.chromosomes) != len(gameta2.chromosomes):
        raise Exception("Геноми мають різну кількість хромосом")
    for i in range(0, len(gameta1.chromosomes)):
        res.chromosomes.append(gameta1.chromosomes[i])
        res.chromosomes.append(gameta2.chromosomes[i])
    return res

In [39]:
new_frog = reproduce(frog1, frog2)
print("Нова жабка")
print(new_frog)

Нова жабка
Геном на 6 хромосоми
Хромосома з 0 пари. На 200 генів
Frog1[0]=0.675253 Frog1[1]=0.315860 Frog1[2]=0.136631 Frog1[3]=0.045811 Frog1[4]=0.482989 ...Frog1[199]=0.942228 
Хромосома з 0 пари. На 200 генів
Frog2[0]=0.945181 Frog2[1]=0.757095 Frog2[2]=0.207437 Frog2[3]=0.598612 Frog2[4]=0.006109 ...Frog2[199]=0.020942 
Хромосома з 1 пари. На 300 генів
Frog1[0]=0.677181 Frog1[1]=0.336475 Frog1[2]=0.372219 Frog1[3]=0.748341 Frog1[4]=0.986958 ...Frog1[299]=0.601624 
Хромосома з 1 пари. На 300 генів
Frog2[0]=0.975290 Frog2[1]=0.822792 Frog2[2]=0.628937 Frog2[3]=0.498362 Frog2[4]=0.957026 ...Frog2[299]=0.734914 
Хромосома з 2 пари. На 400 генів
Frog1[0]=0.344375 Frog1[1]=0.774539 Frog1[2]=0.303013 Frog1[3]=0.553613 Frog1[4]=0.652588 ...Frog1[399]=0.907282 
Хромосома з 2 пари. На 400 генів
Frog2[0]=0.267104 Frog2[1]=0.682288 Frog2[2]=0.905182 Frog2[3]=0.329037 Frog2[4]=0.841249 ...Frog2[399]=0.865342 
...



# Обчислення ступеню подібності

In [44]:
#Процент співпадіння
def match_ratio(owner: str, genome: Genome):
    total = 0
    matched = 0
    for c in genome.chromosomes:
        total += len(c.genes)
        matched += len(list(filter(lambda x: x.owner == owner, c.genes)))
    return matched / total

In [48]:
print("Подібність до жабки 1")
print(match_ratio("Frog1", new_frog))
print("Подібність до жабки 2")
print(match_ratio("Frog2", new_frog))

Подібність до жабки 1
0.5
Подібність до жабки 2
0.5


# Подібність до давніх поколінь

In [51]:
frog11 = mkGenome("Frog1.1", chromosome)
frog12 = mkGenome("Frog1.2", chromosome)
frog51 = reproduce(frog11, frog12)

frog21 = mkGenome("Frog2.1", chromosome)
frog22 = mkGenome("Frog2.2", chromosome)
frog52 = reproduce(frog21, frog22)

frog31 = mkGenome("Frog3.1", chromosome)
frog32 = mkGenome("Frog3.2", chromosome)
frog61 = reproduce(frog31, frog32)

frog41 = mkGenome("Frog4.1", chromosome)
frog42 = mkGenome("Frog4.2", chromosome)
frog62 = reproduce(frog41, frog42)

frog71 = reproduce(frog51, frog52)
frog72 = reproduce(frog61, frog62)

frog81 = reproduce(frog71, frog72)

match_list = ["Frog1.1", "Frog1.2", "Frog2.1", "Frog2.2", "Frog3.1", "Frog3.2", "Frog4.1", "Frog4.2"]
for m in match_list:
    print("Подібність до жабки %s"%m)
    print(match_ratio(m, frog81))



Подібність до жабки Frog1.1
0.21
Подібність до жабки Frog1.2
0.17166666666666666
Подібність до жабки Frog2.1
0.06111111111111111
Подібність до жабки Frog2.2
0.05722222222222222
Подібність до жабки Frog3.1
0.08777777777777777
Подібність до жабки Frog3.2
0.03833333333333333
Подібність до жабки Frog4.1
0.26722222222222225
Подібність до жабки Frog4.2
0.10666666666666667
